In [4]:
'''
Este código carga los datos históricos de precios de Bitcoin desde el cliente de Binance y los procesa 
para la entrada del modelo. Luego, divide los datos en conjuntos de entrenamiento y validación, 
define un modelo de red neuronal LSTM y lo entrena durante 100 épocas utilizando el optimizador 
Adam y la función de pérdida de error cuadrático medio (mse). Finalmente, guarda el modelo 
entrenado en un archivo llamado "btc_price_prediction_model.h5"
'''

'''
TO-DO LIST:
1. Implementar al entrenamiento en el corto-mediano plazo: 
    * la lectura de opiniones en RRSS, 
    * análisis de traders externos 
    * adición de indicadores técnicos
2. Cómo aumentar las redes neuronales?
3. A la hora de graficar los precios de predicción, hay que hacerlo a modo de área predecida y no de precio en especifico 
'''


import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from binance.client import Client

# Autenticación en la API de Binance
api_key = 'T4Bw573BSJCbaHscSo8jn37lE1SOoGsNircF8f7B061WIKBxNkP5nx68vvAev9uk'
api_secret = 'p7VN6HHKR6ZtGkfEeZC3FongJPkR2yr7AjPtPDC8IeaOkASdKZYCKyTuiIfGw57q'
client = Client(api_key, api_secret)

# Obtención de los datos históricos de precios de Bitcoin
klines = client.futures_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "13 year ago UTC") #intervalo de 15minutes a 10 años

# Creación de un dataframe con los datos
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 
                                     'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 
                                     'taker_buy_quote_asset_volume', 'ignore'])

# Eliminación de las columnas que no se van a utilizar
data = data.drop(['timestamp', 'high', 'low', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 
                  'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], axis=1)

# Conversión de los precios a float
data['open'] = data['open'].astype(float)
data['close'] = data['close'].astype(float)

# Normalizar los datos
max_value = data['close'].max()
min_value = data['close'].min()
data['close'] = (data['close'] - min_value) / (max_value - min_value)

# Procesar los datos para la entrada del modelo
data['t-48'] = data['close'].shift(48)
data.dropna(inplace=True)

X = np.array(data[['close', 't-48']])
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
y = np.array(data['close'])

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(X.shape[1], 1)),
    tf.keras.layers.Dense(1)
])

# Compilar el modelo
model.compile(loss='mse', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=20, validation_data=(X_val, y_val))

# Guardar el modelo entrenado
model.save('modelos de entrenamiento/btc_price_prediction_model.h5')

Epoch 1/100


2023-05-13 17:32:50.282675: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-13 17:32:50.283698: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-13 17:32:50.284598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1253/1288 [============================>.] - ETA: 0s - loss: 0.0038

2023-05-13 17:32:52.989073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-13 17:32:52.990069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-13 17:32:52.990968: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1288/1288 [==============================] - 3s 2ms/step - loss: 0.0037 - val_loss: 7.8212e-05
Epoch 2/100
1288/1288 [==============================] - 2s 1ms/step - loss: 6.6277e-05 - val_loss: 5.7096e-05
Epoch 3/100
1288/1288 [==============================] - 2s 1ms/step - loss: 4.2525e-05 - val_loss: 2.9751e-05
Epoch 4/100
1288/1288 [==============================] - 2s 1ms/step - loss: 1.2850e-05 - val_loss: 2.7646e-06
Epoch 5/100
1288/1288 [==============================] - 2s 1ms/step - loss: 2.6458e-06 - val_loss: 1.8970e-06
Epoch 6/100
1288/1288 [==============================] - 2s 1ms/step - loss: 2.7733e-06 - val_loss: 5.2039e-06
Epoch 7/100
1288/1288 [==============================] - 2s 2ms/step - loss: 2.3843e-06 - val_loss: 2.2327e-06
Epoch 8/100
1288/1288 [==============================] - 2s 1ms/step - loss: 2.4296e-06 - val_loss: 2.5153e-06
Epoch 9/100
1288/1288 [==============================] - 2s 1ms/step - loss: 2.0289e-06 - val_loss: 8.1353e-07
Epoch 10/100
1288

-La función train_test_split de Sklearn es utilizada para dividir el conjunto de datos en dos subconjuntos: uno para entrenamiento y otro para prueba. La función toma cuatro argumentos: los datos de entrada (data.iloc[:, :-1]), las etiquetas (data.iloc[:, -1]), el tamaño del conjunto de prueba (test_size=0.2) y la semilla aleatoria (random_state=42) para garantizar la reproducibilidad del experimento. Esta función devuelve cuatro variables: X_train, X_test, y_train y y_test. X_train y y_train son los subconjuntos de entrenamiento, mientras que X_test y y_test son los subconjuntos de prueba.

-tf.keras.Sequential es una función que se utiliza para construir un modelo secuencial, es decir, una pila lineal de capas de redes neuronales. En este caso, se está construyendo un modelo con dos capas densas (Dense). La primera capa tiene 10 unidades, toma una entrada de 4 características (input_shape=[4]) y utiliza la función de activación ReLU (activation='relu'). La segunda capa tiene 3 unidades y utiliza la función de activación Softmax (activation='softmax').

-model.compile se utiliza para compilar el modelo creado. Toma tres argumentos: el optimizador (optimizer=tf.keras.optimizers.Adam(0.01)), la función de pérdida (loss='sparse_categorical_crossentropy') y las métricas (metrics=['accuracy']). En este caso, se utiliza el optimizador Adam con una tasa de aprendizaje de 0.01, la función de pérdida sparse_categorical_crossentropy y la métrica de precisión (accuracy).

-Después de compilar el modelo, se procede a entrenarlo utilizando la función model.fit(). Esta función recibe varios parámetros importantes:

x_train y y_train: son los datos de entrenamiento que se utilizarán para ajustar el modelo.

validation_data: es un conjunto de datos que se utilizará para validar el modelo durante el entrenamiento. En este caso, se utiliza el 20% de los datos de entrenamiento como conjunto de validación.

epochs: es la cantidad de veces que el modelo verá todos los datos de entrenamiento durante el entrenamiento. En este caso, se utilizan 50 épocas.

batch_size: es el número de muestras que se utilizarán para calcular el error y actualizar los pesos del modelo. En este caso, se utilizan 32 muestras por lote.

verbose: es un valor que indica la cantidad de información que se mostrará durante el entrenamiento. En este caso, se utiliza verbose=2, lo que significa que se mostrará una barra de progreso durante el entrenamiento.

callbacks: es una lista de objetos que se utilizarán durante el entrenamiento para realizar acciones específicas en ciertos momentos. En este caso, se utiliza el objeto EarlyStopping para detener el entrenamiento si el modelo deja de mejorar en el conjunto de validación.
